In [364]:
#import

import re
import json
import glob
import os

<h2>AFD e Lexer</h2>
<h4>Nessa parte, definimos a classe de tokens, a classe de AFDs, criamos o afd e por fim definimos a classe lexer</h4>

In [365]:
#class token

class Token:
    def __init__(self, value, type, line):
        self.value = value
        self.type = type
        self.line = line
    
    def __str__(self):
        return f"Token({self.type}, '{self.value}', line {self.line})"
    
    def to_dict(self):
        return {
            "token": self.type,
            "linha": self.line,
            "lexema": self.value
        }

In [366]:
#class afd
class AFD:
    def __init__(self, name, transitions, start, accepts):
        self.name = name
        self.transitions = transitions
        self.start = start
        self.accepts = accepts
        self.reset()

    def reset(self):
        self.state = self.start
        self.lexeme = ""

    def step(self, c):
        trans = self.transitions.get(self.state, {})
        if c in trans:
            self.state = trans[c]
        elif 'LETTER' in trans and c.isalpha():
            self.state = trans['LETTER']
        elif 'DIGIT' in trans and c.isdigit():
            self.state = trans['DIGIT']
        elif 'ANY' in trans and c != '"' and c != "'" and c != '\n':
            self.state = trans['ANY']
        else:
            return False
        self.lexeme += c
        return True

    def is_accepting(self):
        return self.state in self.accepts

    def token_type(self):
        return self.accepts.get(self.state)

In [367]:
#make afd
def make_afd():
    transitions = {
        # Estado inicial (0)
        0: {
            'LETTER': 1, 'DIGIT': 2, '_': 1,
            '=': 3, '!': 4, '>': 5, '<': 6, '-': 7,
            '+': 8, '*': 9, '/': 10,  # Apenas divisão, não comentários
            '(': 11, ')': 12, '{': 13, '}': 14,
            ';': 15, ',': 16, ':': 17,
            '"': 18, "'": 19
        },

        # Identificadores (1)
        1: {'LETTER': 1, 'DIGIT': 1, '_': 1},

        # Números inteiros (2)
        2: {'DIGIT': 2, '.': 20},
        
        # Números float (20 -> 21)
        20: {'DIGIT': 21},
        21: {'DIGIT': 21},

        # Operadores relacionais
        3: {'=': 22},   # = -> ==
        4: {'=': 23},   # ! -> !=
        5: {'=': 24},   # > -> >=
        6: {'=': 25},   # < -> <=
        7: {'>': 26},   # - -> ->

        # Strings (18)
        18: {'ANY': 18, '"': 27},

        # Char literals (19 -> 28 -> 29)
        19: {'ANY': 28},
        28: {"'": 29},
    }

    accepts = {
        1: "ID",
        2: "INT_CONST",
        21: "FLOAT_CONST",
        3: "ASSIGN",
        7: "MINUS",
        8: "PLUS",
        9: "MULT",
        10: "DIV",  # Apenas operador divisão
        22: "EQ",
        23: "NE", 
        24: "GE",
        25: "LE",
        26: "ARROW",
        5: "GT",
        6: "LT",
        11: "LBRACKET",
        12: "RBRACKET",
        13: "LBRACE",
        14: "RBRACE",
        15: "SEMICOLON",
        16: "COMMA",
        17: "COLON",
        27: "FMT_STRING",
        29: "CHAR_LITERAL",
    }

    return AFD("global", transitions, 0, accepts)

In [368]:
#class lexer
class Lexer:
    def __init__(self, source):
        self.source = source
        self.tokens = []
        self.afd = make_afd()

    def tokenize(self):
        i = 0
        line = 1
        
        while i < len(self.source):
            c = self.source[i]

            # Controle de linhas
            if c == '\n':
                line += 1
                i += 1
                continue

            # Ignorar espaços em branco
            if c.isspace():
                i += 1
                continue

            afd = self.afd
            afd.reset()
            j = i
            
            # Processar lexema com AFD
            while j < len(self.source) and afd.step(self.source[j]):
                j += 1

            # Verificar se reconheceu um token
            if afd.is_accepting():
                lex = afd.lexeme
                tipo = afd.token_type()

                # Palavras reservadas
                reserved = {
                    "fn": "FUNCTION", "main": "MAIN", "let": "LET", 
                    "int": "INT", "float": "FLOAT", "char": "CHAR", 
                    "if": "IF", "else": "ELSE", "while": "WHILE", 
                    "println": "PRINTLN", "return": "RETURN"
                }
                
                if lex in reserved:
                    tipo = reserved[lex]

                self.tokens.append(Token(lex, tipo, line))
                i += len(lex)
            else:
                print(f"Erro léxico: caractere inesperado '{self.source[i]}' na linha {line}")
                i += 1
        self.tokens.append(Token("", "EOF", line))
        return self.tokens

<h2>Primeiros testes</h2>
<h4>Aqui executamos os testes referentes a primeira entrega, onde são usados arquivos de teste da pasta "entradas de tokens reconhecidos" e é gerado um json para cada teste que é salvo na pasta "saidas de tokens reconhecidos"</h4>

In [369]:
#teste calculadora.p
entrada = "codigos base da linguagem p/calculadora.p"
saida = "saidas de tokens reconhecidos/calculadora_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

In [370]:
#teste lexical_error.p
entrada = "codigos base da linguagem p/lexical_error.p"
saida = "saidas de tokens reconhecidos/lexical_error_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

Erro léxico: caractere inesperado '[' na linha 3
Erro léxico: caractere inesperado ']' na linha 3
Erro léxico: caractere inesperado '$' na linha 5
Erro léxico: caractere inesperado '&' na linha 6


In [371]:
#teste loop_simples.p
entrada = "codigos base da linguagem p/loop_simples.p"
saida = "saidas de tokens reconhecidos/loop_simples_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

In [372]:
#teste media.p
entrada = "codigos base da linguagem p/media.p"
saida = "saidas de tokens reconhecidos/media_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

In [373]:
#teste soma.p
entrada = "codigos base da linguagem p/soma.p"
saida = "saidas de tokens reconhecidos/soma_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

In [374]:
#teste tokens.p
entrada = "codigos base da linguagem p/tokens.p"
saida = "saidas de tokens reconhecidos/tokens_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

<h2>Parser e Tabela de simbolos</h2>

In [375]:
#class parser

class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0
        self.current_token = tokens[0] if tokens else None
        self.errors = []

    def parse(self, nome_arquivo=None):
        try:
            self.parse_programa()
            if self.current_token and self.current_token.type != "EOF":
                self.error("Esperado fim de arquivo")
        except Exception as e:
            self.error(f"Erro de sintaxe: {str(e)}")
            
        self.generate_output_files(nome_arquivo)

    def advance(self):
        self.pos += 1
        if self.pos < len(self.tokens):
            self.current_token = self.tokens[self.pos]
        else:
            self.current_token = None
    
    def match(self, expected_type):
        if self.current_token is None:
            self.error(f"Esperado '{expected_type}', encontrado 'EOF'")
            return None
        if self.current_token.type == expected_type:
            token = self.current_token
            self.advance()
            return token
        else:
            found = self.current_token.type 
            self.error(f"Esperado '{expected_type}', encontrado '{found}'")
            return None
    
    def error(self, message):
        if self.current_token is not None:
            line = self.current_token.line
        elif self.tokens:
            line = self.tokens[-1].line
        else:
            line = 1
        
        error_message = f"Linha {line}: {message}"
        self.errors.append(error_message)
        print(f"Erro de sintaxe: {error_message}")
    
    # ========== MÉTODOS DE RECUPERAÇÃO DE ERROS ==========
    def sync_to(self, sync_tokens):
        """Recuperação de erro - pula tokens até encontrar um dos sync_tokens"""
        while (self.current_token and 
               self.current_token.type not in sync_tokens and
               self.current_token.type != "EOF"):
            print(f"  [RECUPERAÇÃO] Pulando token: {self.current_token}")
            self.advance()
        
        if self.current_token and self.current_token.type in sync_tokens:
            print(f"  [RECUPERAÇÃO] Sincronizado em: {self.current_token}")
            return True
        return False

    def match_with_recovery(self, expected_type, sync_tokens=None):
        """Match com recuperação de erro"""
        if sync_tokens is None:
            sync_tokens = ["SEMICOLON", "RBRACE", "FUNCTION", "EOF"]
            
        if self.current_token is None:
            self.error(f"Esperado '{expected_type}', encontrado 'EOF'")
            return None
            
        if self.current_token.type == expected_type:
            token = self.current_token
            self.advance()
            return token
        else:
            found = self.current_token.type 
            self.error(f"Esperado '{expected_type}', encontrado '{found}'")
            # Recuperação
            self.sync_to(sync_tokens)
            return None
    
    # ========== PRODUÇÕES DA GRAMÁTICA ==========
    
    def parse_programa(self):
        self.parse_funcao()
        self.parse_funcao_seq()
    
    def parse_funcao_seq(self):
        while self.current_token and self.current_token.type == "FUNCTION":
            self.parse_funcao()

    def parse_funcao(self):
        self.match_with_recovery("FUNCTION")
        self.parse_nome_funcao()
        self.match_with_recovery("LBRACKET")
        self.parse_lista_params()
        self.match_with_recovery("RBRACKET")
        self.parse_tipo_retorno_funcao()
        self.parse_bloco()
    
    def parse_nome_funcao(self):
        if self.current_token and self.current_token.type in ["MAIN", "ID"]:
            return self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado nome de função (ID ou 'main')")
            return None
    
    def parse_lista_params(self):
        if self.current_token and self.current_token.type == "ID":
            self.match_with_recovery("ID")
            self.match_with_recovery("COLON")
            self.parse_type()
            self.parse_lista_params2()
        # Senão, é ε - não faz nada
    
    def parse_lista_params2(self):
        while self.current_token and self.current_token.type == "COMMA":
            self.match_with_recovery("COMMA")
            self.match_with_recovery("ID")
            self.match_with_recovery("COLON")
            self.parse_type()
    
    def parse_tipo_retorno_funcao(self):
        if self.current_token and self.current_token.type == "ARROW":
            self.match_with_recovery("ARROW")
            self.parse_type()
        # Senão, é ε - não faz nada

    def parse_type(self):
        if self.current_token and self.current_token.type in ["INT", "FLOAT", "CHAR"]:
            return self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado tipo de dado (int, float ou char)")
            return None
    
    def parse_bloco(self):
        self.match_with_recovery("LBRACE")
        self.parse_sequencia()
        self.match_with_recovery("RBRACE")
    
    def parse_sequencia(self):
        while self.current_token and self.current_token.type in self._first_declaracao_comando():
            if self._is_declaracao():
                self.parse_declaracao()
            else:
                self.parse_comando()
    
    def _first_declaracao_comando(self):
        return ["LET", "ID", "IF", "WHILE", "PRINTLN", "RETURN"]
    
    def _is_declaracao(self):
        return self.current_token and self.current_token.type == "LET"
    
    # ========== SISTEMA DE SAÍDA ==========
    def generate_output_files(self, nome_arquivo=None):
        """Gera arquivos de saída unificados"""
        self._generate_syntax_errors(nome_arquivo)

    def _generate_syntax_errors(self, nome_arquivo=None): 
        try:
            os.makedirs("saidas do analisador sintatico", exist_ok=True)
            
            # Se não tem nome_arquivo, usa o padrão
            if nome_arquivo is None:
                nome_base = "syntax_errors"
            else:
                # Remove extensão .p se existir
                nome_base = os.path.splitext(nome_arquivo)[0]
            
            # Arquivo JSON específico para este arquivo
            json_path = f"saidas do analisador sintatico/{nome_base}_syntax.json"
            
            with open(json_path, "w", encoding="utf-8") as f:
                resultado = {
                    "arquivo": nome_arquivo or "desconhecido",
                    "valido": len(self.errors) == 0,
                    "total_erros": len(self.errors),
                    "erros": self.errors
                }
                json.dump(resultado, f, indent=2, ensure_ascii=False)
                        
        except Exception as e:
            print(f"Erro ao gerar arquivo de erros de sintaxe: {str(e)}")
    
    def parse_declaracao(self):
        self.match_with_recovery("LET")
        self.parse_var_list()
        self.match_with_recovery("COLON")
        self.parse_type()
        self.match_with_recovery("SEMICOLON")
    
    def parse_var_list(self):
        self.match_with_recovery("ID")
        self.parse_var_list2()

    def parse_var_list2(self):
        while self.current_token and self.current_token.type == "COMMA":
            self.match_with_recovery("COMMA")
            self.match_with_recovery("ID")
    
    def parse_comando(self):
        if self.current_token and self.current_token.type == "ID":
            # Lookahead sem consumir
            next_pos = self.pos + 1
            if next_pos < len(self.tokens) and self.tokens[next_pos].type == "ASSIGN":
                # É atribuição: ID = Expr ;
                self.match_with_recovery("ID")
                self.match_with_recovery("ASSIGN")
                self.parse_expr()
                self.match_with_recovery("SEMICOLON")
            elif next_pos < len(self.tokens) and self.tokens[next_pos].type == "LBRACKET":
                # É chamada: ID ( ListaArgs ) ;
                self.match_with_recovery("ID")
                self.match_with_recovery("LBRACKET")
                self.parse_lista_args()
                self.match_with_recovery("RBRACKET")
                self.match_with_recovery("SEMICOLON")
            else:
                self.error("Esperado '=' ou '(' após identificador")
                self.sync_to(["SEMICOLON"])
        elif self.current_token and self.current_token.type == "IF":
            self.parse_comando_se()
        elif self.current_token and self.current_token.type == "WHILE":
            self.parse_comando_while()
        elif self.current_token and self.current_token.type == "PRINTLN":
            self.parse_comando_println()
        elif self.current_token and self.current_token.type == "RETURN":
            self.parse_comando_return()
        else:
            self.error("Comando inválido")
            self.sync_to(["SEMICOLON", "RBRACE"])
    
    def parse_comando_se(self):
        """ComandoSe → if Expr Bloco ComandoSenao"""
        self.match_with_recovery("IF")
        self.parse_expr()  
        self.parse_bloco()
        self.parse_comando_senao()
    
    def parse_comando_senao(self):
        """ComandoSenao → else ComandoSe | ε"""
        if self.current_token and self.current_token.type == "ELSE":
            self.match_with_recovery("ELSE")
            # Pode ser outro if ou um bloco simples
            if self.current_token and self.current_token.type == "IF":
                self.parse_comando_se()
            else:
                self.parse_bloco()
        # Senão, é ε - não faz nada
    
    def parse_comando_while(self):
        """ComandoWhile → while Expr Bloco"""
        self.match_with_recovery("WHILE")
        self.parse_expr()
        self.parse_bloco()
        
    def parse_comando_println(self):
        self.match_with_recovery("PRINTLN")
        self.match_with_recovery("LBRACKET")
        self.match_with_recovery("FMT_STRING")
        self.match_with_recovery("COMMA")
        self.parse_lista_args()
        self.match_with_recovery("RBRACKET")
        self.match_with_recovery("SEMICOLON")
    
    def parse_comando_return(self):
        self.match_with_recovery("RETURN")
        self.parse_expr()
        self.match_with_recovery("SEMICOLON")
    
    def parse_lista_args(self):
        if self.current_token and self.current_token.type in ["ID", "INT_CONST", "FLOAT_CONST", "CHAR_LITERAL"]:
            self.parse_arg()
            self.parse_lista_args2()
        # Senão, é ε - não faz nada
        
    def parse_lista_args2(self):
        while self.current_token and self.current_token.type == "COMMA":
            self.match_with_recovery("COMMA")
            self.parse_arg()
    
    def parse_arg(self):
        if self.current_token and self.current_token.type == "ID":
            # Lookahead para ver se é chamada de função
            next_pos = self.pos + 1
            if next_pos < len(self.tokens) and self.tokens[next_pos].type == "LBRACKET":
                self.match_with_recovery("ID")
                self.match_with_recovery("LBRACKET")
                self.parse_lista_args()
                self.match_with_recovery("RBRACKET")
            else:
                # Apenas uma variável
                self.match_with_recovery("ID")
        elif self.current_token and self.current_token.type in ["INT_CONST", "FLOAT_CONST", "CHAR_LITERAL"]:
            self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado ID, constante ou literal")
            # Recuperação: avança para evitar loop
            if self.current_token:
                self.advance()
    
    def parse_chamada_funcao(self):
        if self.current_token and self.current_token.type == "LBRACKET":
            self.match_with_recovery("LBRACKET")
            self.parse_lista_args()
            self.match_with_recovery("RBRACKET")
        # Senão, é ε - não faz nada
    
    # ========== EXPRESSÕES ==========
    def parse_expr(self):
        """Expr → Rel ExprOpc"""
        self.parse_rel()
        self.parse_expr_opc()

    def parse_expr_opc(self):
        """ExprOpc → OpIgual Rel ExprOpc | ε"""
        if self.current_token and self.current_token.type in ["EQ", "NE"]:
            self.parse_op_igual()
            self.parse_rel()
            self.parse_expr_opc()
        # Senão, é ε

    def parse_op_igual(self):
        if self.current_token and self.current_token.type in ["EQ", "NE"]:
            self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado '==' ou '!='")

    def parse_rel(self):
        """Rel → Adicao RelOpc"""
        self.parse_adicao()
        self.parse_rel_opc()

    def parse_rel_opc(self):
        """RelOpc → OpRel Adicao RelOpc | ε"""
        if self.current_token and self.current_token.type in ["LT", "LE", "GT", "GE"]:
            self.parse_op_rel()
            self.parse_adicao()
            self.parse_rel_opc()
        # Senão, é ε

    def parse_op_rel(self):
        if self.current_token and self.current_token.type in ["LT", "LE", "GT", "GE"]:
            self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado operador relacional")

    def parse_adicao(self):
        """Adicao → Termo AdicaoOpc"""
        self.parse_termo()
        self.parse_adicao_opc()

    def parse_adicao_opc(self):
        """AdicaoOpc → OpAdicao Termo AdicaoOpc | ε"""
        while self.current_token and self.current_token.type in ["PLUS", "MINUS"]:
            self.parse_op_adicao()
            self.parse_termo()

    def parse_op_adicao(self):
        if self.current_token and self.current_token.type in ["PLUS", "MINUS"]:
            self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado '+' ou '-'")

    def parse_termo(self):
        """Termo → Fator TermoOpc"""
        self.parse_fator()
        self.parse_termo_opc()

    def parse_termo_opc(self):
        """TermoOpc → OpMult Fator TermoOpc | ε"""
        while self.current_token and self.current_token.type in ["MULT", "DIV"]:
            self.parse_op_mult()
            self.parse_fator()

    def parse_op_mult(self):
        if self.current_token and self.current_token.type in ["MULT", "DIV"]:
            self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado '*' ou '/'")

    def parse_fator(self):
        """Fator → ID ChamadaFuncao | CONST | LITERAL | ( Expr )"""
        if self.current_token and self.current_token.type == "ID":
            self.match_with_recovery("ID")
            self.parse_chamada_funcao()
        elif self.current_token and self.current_token.type in ["INT_CONST", "FLOAT_CONST", "CHAR_LITERAL"]:
            self.match_with_recovery(self.current_token.type)
        elif self.current_token and self.current_token.type == "LBRACKET":
            self.match_with_recovery("LBRACKET")
            self.parse_expr()
            self.match_with_recovery("RBRACKET")
        else:
            self.error("Esperado ID, constante, literal ou '('")
            # Recuperação: avança para evitar loop
            if self.current_token:
                self.advance()

In [376]:
def testar_arquivos_base_completo():
    """Testa todos os arquivos .p da pasta base com o parser corrigido"""
    
    arquivos = glob.glob("codigos base da linguagem p/*.p")
    
    print("=== 📁 TESTANDO ARQUIVOS BASE COM PARSER CORRIGIDO ===\n")
    
    resultados = []
    
    for arquivo_path in arquivos:
        nome_arquivo = os.path.basename(arquivo_path)
        print(f"📂 {nome_arquivo:25}", end="")
        
        try:
            with open(arquivo_path, "r", encoding="utf-8") as f:
                codigo = f.read()
            
            lexer = Lexer(codigo)
            tokens = lexer.tokenize()
            parser = Parser(tokens)
            parser.parse(nome_arquivo)  # ← PASSA O NOME DO ARQUIVO
            
            erros_count = len(parser.errors)
            valido = erros_count == 0
            
            if valido:
                print("✅ VÁLIDO")
            else:
                print(f"❌ {erros_count} erro(s)")
                # Mostra os primeiros erros
                for erro in parser.errors[:2]:  # Mostra até 2 erros por arquivo
                    print(f"     {erro}")
            
            resultados.append({
                'arquivo': nome_arquivo,
                'valido': valido,
                'erros': erros_count,
                'mensagens_erro': parser.errors[:5]  # Guarda até 5 erros
            })
            
        except Exception as e:
            print(f"💥 ERRO: {e}")
            resultados.append({
                'arquivo': nome_arquivo,
                'valido': False,
                'erro_excecao': str(e)
            })
    
    # Resumo detalhado (FORA DO LOOP)
    print(f"\n{'='*50}")
    print("📈 RESUMO DETALHADO:")
    print(f"{'='*50}")
    
    validos = sum(1 for r in resultados if r.get('valido', False))
    totais = len(arquivos)
    
    print(f"Arquivos válidos: {validos}/{totais} ({validos/totais*100:.1f}%)")
    print()
    
    # Lista arquivos válidos
    print("✅ ARQUIVOS VÁLIDOS:")
    for r in resultados:
        if r.get('valido', False):
            print(f"   ✓ {r['arquivo']}")
    
    print()
    
    # Lista arquivos com erros
    print("❌ ARQUIVOS COM ERROS:")
    for r in resultados:
        if not r.get('valido', False):
            print(f"   ✗ {r['arquivo']}")
            if 'erros' in r:
                print(f"      {r['erros']} erro(s) sintático(s)")
            if 'erro_excecao' in r:
                print(f"      Erro de exceção: {r['erro_excecao']}")
            if 'mensagens_erro' in r and r['mensagens_erro']:
                for erro in r['mensagens_erro'][:2]:  # Mostra até 2 erros
                    print(f"      - {erro}")
            print()
    
    # Salvar resultados em JSON
    try:
        os.makedirs("saidas do analisador sintatico", exist_ok=True)
        with open("saidas do analisador sintatico/resultados_testes.json", "w", encoding="utf-8") as f:
            json.dump(resultados, f, indent=2, ensure_ascii=False)
        print(f"💾 Resultados salvos em: saidas do analisador sintatico/resultados_testes.json")
    except Exception as e:
        print(f"⚠️  Não foi possível salvar os resultados: {e}")
    
    return resultados

# Executar teste completo
print("Iniciando teste de todos os arquivos base...\n")
resultados_finais = testar_arquivos_base_completo()

Iniciando teste de todos os arquivos base...

=== 📁 TESTANDO ARQUIVOS BASE COM PARSER CORRIGIDO ===

📂 loop_simples.p           ✅ VÁLIDO
📂 media.p                  ✅ VÁLIDO
📂 soma.p                   ✅ VÁLIDO
📂 calculadora.p            ✅ VÁLIDO
📂 tokens.p                 Erro de sintaxe: Linha 1: Esperado 'LBRACKET', encontrado 'LET'
  [RECUPERAÇÃO] Pulando token: Token(LET, 'let', line 1)
  [RECUPERAÇÃO] Pulando token: Token(INT, 'int', line 1)
  [RECUPERAÇÃO] Pulando token: Token(FLOAT, 'float', line 1)
  [RECUPERAÇÃO] Pulando token: Token(CHAR, 'char', line 1)
  [RECUPERAÇÃO] Pulando token: Token(IF, 'if', line 1)
  [RECUPERAÇÃO] Pulando token: Token(ELSE, 'else', line 1)
  [RECUPERAÇÃO] Pulando token: Token(WHILE, 'while', line 1)
  [RECUPERAÇÃO] Pulando token: Token(PRINTLN, 'println', line 1)
  [RECUPERAÇÃO] Pulando token: Token(RETURN, 'return', line 1)
  [RECUPERAÇÃO] Pulando token: Token(LBRACKET, '(', line 3)
  [RECUPERAÇÃO] Pulando token: Token(RBRACKET, ')', line 3)
  [RECU